<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/cover-small.jpg">

*Este libro es una versión al español de [Python for Everybody](https://www.py4e.com/) escrito por el [Dr. Charles R. Severance](http://www.dr-chuck.com/); este contenido esta disponible en [GitHub](https://github.com/csev/py4e).*

Detalles de Copyright

*Copyright ~ 2009- Charles Severance.
Este trabajo está registrado bajo una Licencia Creative Commons AttributionNonCommercial-ShareAlike 3.0 [CC BY-NC-SA](https://creativecommons.org/licenses/by-nc-sa/3.0/).*

<!--NAVIGATION-->
| [Indice](indice.ipynb) | 

< [Capítulo 15 - Uso de bases de datos y SQL](cap15.ipynb) |

# Capítulo 16 - Visualización de datos

Hasta ahora hemos estado aprendiendo el lenguaje Python y luego aprendiendo cómo usar Python, la red y las bases de datos para manipular datos.

En este capítulo, analizamos tres aplicaciones completas que reúnen todas estas cosas para administrar y visualizar datos. Puede usar estas aplicaciones como código de muestra para ayudarlo a comenzar a resolver un problema del mundo real.

Cada una de las aplicaciones es un archivo ZIP que puede descargar y extraer en su computadora y ejecutar.

## Crear un mapa de Google a partir de datos geocodificados

En este proyecto, estamos utilizando la API de geocodificación de Google para limpiar algunas ubicaciones geográficas ingresadas por los usuarios de los nombres de las universidades y luego colocar los datos en un mapa de Google.

![Figura 16.1](./figures/google-map.png)
<center><i>Figura 16.1: Mapa de Google.</i></center>

Para comenzar, descargue la aplicación desde: www.py4e.com/code3/geodata.zip

El primer problema a resolver es que la API gratuita de geocodificación de Google está limitada a una determinada cantidad de solicitudes por día. Si tiene muchos datos, es posible que deba detener y reiniciar el proceso de búsqueda varias veces. Entonces dividimos el problema en dos fases.

En la primera fase tomamos nuestros datos de "encuesta" de entrada en el archivo `where.data` y lo leemos una línea a la vez, y recuperamos la información geocodificada de Google y la almacenamos en una base de datos `geodata.sqlite`. Antes de usar la API de geocodificación para cada ubicación ingresada por el usuario, simplemente verificamos si ya tenemos los datos para esa línea de entrada en particular. La base de datos funciona como un "caché" local de nuestros datos de geocodificación para garantizar que nunca le solicitemos a Google los mismos datos dos veces.

Puede reiniciar el proceso en cualquier momento eliminando el archivo `geodata.sqlite`.

Ejecute el programa `geoload.py`. Este programa leerá las líneas de entrada en `where.data` y para cada línea verifique si ya está en la base de datos. Si no tenemos los datos para la ubicación, llamará a la API de geocodificación para recuperar los datos y almacenarlos en la base de datos.

Aquí hay una muestra de ejecución después de que ya hay algunos datos en la base de datos:

    Found in database  Northeastern University
    Found in database  University of Hong Kong, ...
    Found in database  Technion
    Found in database  Viswakarma Institute, Pune, India
    Found in database  UMD
    Found in database  Tufts University

    Resolving Monash University
    Retrieving http://maps.googleapis.com/maps/api/
        geocode/json?address=Monash+University
    Retrieved 2063 characters {    "results" : [
    {'status': 'OK', 'results': ... }

    Resolving Kokshetau Institute of Economics and Management
    Retrieving http://maps.googleapis.com/maps/api/
        geocode/json?address=Kokshetau+Inst ...
    Retrieved 1749 characters {    "results" : [
    {'status': 'OK', 'results': ... }
    ...

Las primeras cinco ubicaciones ya están en la base de datos y, por lo tanto, se omiten. El programa escanea hasta el punto donde encuentra nuevas ubicaciones y comienza a recuperarlas.

El programa `geoload.py` se puede detener en cualquier momento, y hay un contador que puede usar para limitar la cantidad de llamadas a la API de geocodificación para cada ejecución. Dado que `where.data` solo tiene unos pocos cientos de elementos de datos, no debe ejecutar el límite de velocidad diario, pero si tuviera más datos, podría tomar varias ejecuciones durante varios días para que su base de datos tenga todos los datos geocodificados para tu aportación.

Una vez que tenga algunos datos cargados en `geodata.sqlite`, puede visualizar los datos usando el programa `geodump.py`. Este programa lee la base de datos y escribe el archivo `where.js` con la ubicación, latitud y longitud en forma de código JavaScript ejecutable.

Una ejecución del programa `geodump.py` es la siguiente:

    Northeastern University, ... Boston, MA 02115, USA 42.3396998 -71.08975
    Bradley University, 1501 ... Peoria, IL 61625, USA 40.6963857 -89.6160811
    ...
    Technion, Viazman 87, Kesalsaba, 32000, Israel 32.7775 35.0216667
    Monash University Clayton ... VIC 3800, Australia -37.9152113 145.134682
    Kokshetau, Kazakhstan 53.2833333 69.3833333
    ...
    12 records written to where.js
    Open where.html to view the data in a browser

El archivo `where.html` consta de HTML y JavaScript para visualizar un mapa de Google. Lee los datos más recientes en `where.js` para obtener los datos que se visualizarán. Aquí está el formato del archivo `where.js`:

    myData = [
    [42.3396998,-71.08975, 'Northeastern Uni ... Boston, MA 02115'],
    [40.6963857,-89.6160811, 'Bradley University, ... Peoria, IL 61625, USA'],
    [32.7775,35.0216667, 'Technion, Viazman 87, Kesalsaba, 32000, Israel'],
       ...
    ];

Esta es una variable de JavaScript que contiene una lista de listas. La sintaxis para las constantes de la lista de JavaScript es muy similar a Python, por lo que la sintaxis debería serle familiar.

Simplemente abra `where.html` en un navegador para ver las ubicaciones. Puede pasar el cursor sobre cada pin del mapa para encontrar la ubicación que la API de geocodificación devolvió para la entrada ingresada por el usuario. Si no puede ver ningún dato al abrir el archivo `where.html`, puede consultar el JavaScript o la consola del desarrollador de su navegador.

## Visualización de redes e interconexiones

En esta aplicación, realizaremos algunas de las funciones de un motor de búsqueda. Primero analizaremos un pequeño subconjunto de la web y ejecutaremos una versión simplificada del algoritmo de clasificación de páginas de Google para determinar qué páginas están más conectadas, y luego visualizaremos la clasificación de páginas y la conectividad de nuestro pequeño rincón de la web. Utilizaremos la biblioteca de visualización D3 JavaScript http://d3js.org/ para producir la salida de visualización.

Puede descargar y extraer esta aplicación desde: www.py4e.com/code3/pagerank.zip.

![Figura 16.2](./figures/pagerank.png)
<center><i>Figura 16.2: Un ranking de una página.</i></center>

El primer programa `spider.py` rastrea un sitio web y extrae una serie de páginas a la base de datos `spider.sqlite`, registrando los enlaces entre las páginas. Puede reiniciar el proceso en cualquier momento eliminando el archivo `spider.sqlite` y volviendo a ejecutar `spider.py`.

    Enter web url or enter: http://www.dr-chuck.com/
    ['http://www.dr-chuck.com']
    How many pages:2
    1 http://www.dr-chuck.com/ 12
    2 http://www.dr-chuck.com/csev-blog/ 57
    How many pages

En este ejemplo, le dijimos que rastreara un sitio web y recuperara dos páginas. Si reinicia el programa y le dice que rastree más páginas, no volverá a rastrear ninguna página que ya esté en la base de datos. Al reiniciar, va a una página aleatoria no rastreada y comienza allí. Por lo tanto, cada ejecución sucesiva de `spider.py` es aditiva.

    Enter web url or enter: http://www.dr-chuck.com/
    ['http://www.dr-chuck.com']
    How many pages:3
    3 http://www.dr-chuck.com/csev-blog 57
    4 http://www.dr-chuck.com/dr-chuck/resume/speaking.htm 1
    5 http://www.dr-chuck.com/dr-chuck/resume/index.htm 13
    How many pages:

Puede tener múltiples puntos de partida en la misma base de datos; dentro del programa, estos se denominan "webs". La araña elige aleatoriamente entre todos los enlaces no visitados en todas las webs como la página siguiente a la araña.

Si desea volcar el contenido del archivo `spider.sqlite`, puede ejecutar `spdump.py` de la siguiente manera:

    (5, None, 1.0, 3, 'http://www.dr-chuck.com/csev-blog')
    (3, None, 1.0, 4, 'http://www.dr-chuck.com/dr-chuck/resume/speaking.htm')
    (1, None, 1.0, 2, 'http://www.dr-chuck.com/csev-blog/')
    (1, None, 1.0, 5, 'http://www.dr-chuck.com/dr-chuck/resume/index.htm')
    4 rows.

Esto muestra el número de enlaces entrantes, el rango de página anterior, el nuevo rango de página, la identificación de la página y la URL de la página. El programa `spdump.py` solo muestra páginas que tienen al menos un enlace entrante.

Una vez que tenga algunas páginas en la base de datos, puede ejecutar el rango de página en las páginas usando el programa `sprank.py`. Simplemente dígale cuántas iteraciones de rango de página ejecutar.

    How many iterations:2
    1 0.546848992536
    2 0.226714939664
    [(1, 0.559), (2, 0.659), (3, 0.985), (4, 2.135), (5, 0.659)]

Puede volcar la base de datos nuevamente para ver que el rango de página se ha actualizado:

    (5, 1.0, 0.985, 3, 'http://www.dr-chuck.com/csev-blog')
    (3, 1.0, 2.135, 4, 'http://www.dr-chuck.com/dr-chuck/resume/speaking.htm')
    (1, 1.0, 0.659, 2, 'http://www.dr-chuck.com/csev-blog/')
    (1, 1.0, 0.659, 5, 'http://www.dr-chuck.com/dr-chuck/resume/index.htm')
    4 rows.
    
Puede ejecutar `sprank.py` tantas veces como desee y simplemente refinará el rango de página cada vez que lo ejecute. Incluso puede ejecutar `sprank.py` varias veces y luego ir a spider algunas páginas más con `spider.py` y luego ejecutar `sprank.py` para volver a converger los valores de rango de página. Un motor de búsqueda generalmente ejecuta los programas de rastreo y clasificación todo el tiempo.

Si desea reiniciar los cálculos de rango de página sin desplazar las páginas web, puede usar `spreset.py` y luego reiniciar `sprank.py`.

    How many iterations:50
    1 0.546848992536
    2 0.226714939664
    3 0.0659516187242
    4 0.0244199333
    5 0.0102096489546
    6 0.00610244329379
    ...
    42 0.000109076928206
    43 9.91987599002e-05
    44 9.02151706798e-05
    45 8.20451504471e-05
    46 7.46150183837e-05
    47 6.7857770908e-05
    48 6.17124694224e-05
    49 5.61236959327e-05
    50 5.10410499467e-05
    [(512, 0.0296), (1, 12.79), (2, 28.93), (3, 6.808), (4, 13.46)]

Para cada iteración del algoritmo de rango de página, imprime el cambio promedio en el rango de página por página. Inicialmente, la red está bastante desequilibrada y, por lo tanto, los valores individuales de rango de página cambian enormemente entre iteraciones. Pero en algunas iteraciones cortas, el rango de la página converge. Debe ejecutar `sprank.py` el tiempo suficiente para que converjan los valores de rango de página.

Si desea visualizar las páginas principales actuales en términos de rango de página, ejecute `spjson.py` para leer la base de datos y escriba los datos de las páginas más vinculadas en formato JSON para verlas en un navegador web.

    Creating JSON output on spider.json...
    How many nodes? 30
    Open force.html in a browser to view the visualization

Puede ver estos datos abriendo el archivo `force.html` en su navegador web. Esto muestra un diseño automático de los nodos y enlaces. Puede hacer clic y arrastrar cualquier nodo y también puede hacer doble clic en un nodo para encontrar la URL que está representada por el nodo.

Si vuelve a ejecutar las otras utilidades, vuelva a ejecutar `spjson.py` y presione actualizar en el navegador para obtener los nuevos datos de `spider.json`.

## Visualizar datos de correo

Hasta este punto en el libro, te has familiarizado bastante con nuestros archivos de datos `mbox-short.txt` y `mbox.txt`. Ahora es el momento de llevar nuestro análisis de datos de correo electrónico al siguiente nivel.

En el mundo real, a veces hay que extraer los datos de correo de los servidores. Eso podría llevar bastante tiempo y los datos podrían ser inconsistentes, estar llenos de errores y necesitar mucha limpieza o ajuste. En esta sección, trabajamos con una aplicación que es la más compleja hasta ahora y extraemos casi un gigabyte de datos y los visualizamos.

![Figura 16.3](./figures/wordcloud.png)
<center><i>Figura 16.3: Una nube de palabras de la lista de desarrolladores de Sakai</i></center>

Puede descargar esta aplicación desde: www.py4e.com/code3/gmane.zip.

Utilizaremos datos de un servicio gratuito de archivo de listas de correo electrónico llamado www.gmane.org . Este servicio es muy popular entre los proyectos de código abierto porque proporciona un buen archivo de búsqueda de su actividad de correo electrónico. También tienen una política muy liberal con respecto al acceso a sus datos a través de su API. No tienen límites de velocidad, pero solicite que no sobrecargue su servicio y tome solo los datos que necesita. Puede leer los términos y condiciones de gmane en esta página: http://gmane.org/export.php.

Es muy importante que haga uso responsable de los datos de gmane.org agregando demoras a su acceso a sus servicios y extendiendo los trabajos de larga duración durante un período de tiempo más largo. No abuse de este servicio gratuito y lo arruine para el resto de nosotros.

Cuando los datos de correo electrónico de Sakai se procesaron utilizando este software, produjeron casi un Gigabyte de datos y tomaron varias ejecuciones en varios días. El archivo `README.txt` en el ZIP anterior puede tener instrucciones sobre cómo puede descargar una copia previamente dividida del archivo `content.sqlite` para la mayoría del corpus de correo electrónico de Sakai para que no tenga que arañar durante cinco días solo para ejecutar los programas Si descarga el contenido pre-spidered, aún debe ejecutar el proceso de spidering para ponerse al día con los mensajes más recientes.

El primer paso es arañar el repositorio gmane. La URL base está codificada en `gmane.py` y está codificada en la lista de desarrolladores de Sakai. Puede arañar otro repositorio cambiando esa url base. Asegúrese de eliminar el archivo `content.sqlite` si cambia la url base.

El archivo `gmane.py` funciona como una araña de almacenamiento en caché responsable, ya que se ejecuta lentamente y recupera un mensaje de correo por segundo para evitar ser estrangulado por gmane. Almacena todos sus datos en una base de datos y se puede interrumpir y reiniciar tantas veces como sea necesario. Puede llevar muchas horas extraer todos los datos. Por lo tanto, es posible que deba reiniciar varias veces.

Aquí hay una ejecución de `gmane.py` recuperando los últimos cinco mensajes de la lista de desarrolladores de Sakai:

    How many messages:10
    http://download.gmane.org/gmane.comp.cms.sakai.devel/51410/51411 9460
        nealcaidin@sakaifoundation.org 2013-04-05 re: [building ...
    http://download.gmane.org/gmane.comp.cms.sakai.devel/51411/51412 3379
        samuelgutierrezjimenez@gmail.com 2013-04-06 re: [building ...
    http://download.gmane.org/gmane.comp.cms.sakai.devel/51412/51413 9903
        da1@vt.edu 2013-04-05 [building sakai] melete 2.9 oracle ...
    http://download.gmane.org/gmane.comp.cms.sakai.devel/51413/51414 349265
        m.shedid@elraed-it.com 2013-04-07 [building sakai] ...
    http://download.gmane.org/gmane.comp.cms.sakai.devel/51414/51415 3481
        samuelgutierrezjimenez@gmail.com 2013-04-07 re: ...
    http://download.gmane.org/gmane.comp.cms.sakai.devel/51415/51416 0

    Does not start with From

El programa escanea `content.sqlite` desde uno hasta el primer número de mensaje que aún no se ha arañado y comienza a arañar ese mensaje. Continúa arañando hasta que ha arañado el número deseado de mensajes o llega a una página que no parece ser un mensaje con el formato correcto.

A veces a gmane.org le falta un mensaje. Quizás los administradores puedan eliminar mensajes o quizás se pierdan. Si su araña se detiene, y parece que ha encontrado un mensaje faltante, vaya al Administrador de SQLite y agregue una fila con la identificación faltante dejando todos los demás campos en blanco y reinicie `gmane.py`. Esto despegará el proceso de arrastre y permitirá que continúe. Estos mensajes vacíos serán ignorados en la siguiente fase del proceso.

Una cosa buena es que una vez que haya revisado todos los mensajes y los tenga en `content.sqlite`, puede ejecutar `gmane.py` nuevamente para obtener nuevos mensajes a medida que se envían a la lista.

Los datos de `content.sqlite` son bastante crudos, con un modelo de datos ineficiente y no están comprimidos. Esto es intencional ya que le permite mirar `content.sqlite` en el Administrador de SQLite para depurar problemas con el proceso de spidering. Sería una mala idea ejecutar consultas en esta base de datos, ya que serían bastante lentas.

El segundo proceso es ejecutar el programa `gmodel.py`. Este programa lee los datos sin procesar de `content.sqlite` y produce una versión limpia y modelada de los datos en el archivo `index.sqlite`. Este archivo será mucho más pequeño (a menudo 10 veces más pequeño) que `content.sqlite` porque también comprime el encabezado y el texto del cuerpo.

Cada vez que `gmodel.py` se ejecuta, elimina y reconstruye `index.sqlite`, lo que le permite ajustar sus parámetros y editar las tablas de mapeo en `content.sqlite` para modificar el proceso de limpieza de datos. Esta es una muestra de ejecución de `gmodel.py`. Imprime una línea cada vez que se procesan 250 mensajes de correo para que pueda ver algún progreso, ya que este programa puede ejecutarse durante un tiempo procesando casi un gigabyte de datos de correo.

    Loaded allsenders 1588 and mapping 28 dns mapping 1
    1 2005-12-08T23:34:30-06:00 ggolden22@mac.com
    251 2005-12-22T10:03:20-08:00 tpamsler@ucdavis.edu
    501 2006-01-12T11:17:34-05:00 lance@indiana.edu
    751 2006-01-24T11:13:28-08:00 vrajgopalan@ucmerced.edu
    ...

El programa `gmodel.py` maneja una serie de tareas de limpieza de datos.

Los nombres de dominio se truncan a dos niveles para .com, .org, .edu y .net. Otros nombres de dominio se truncan a tres niveles. Entonces si.umich.edu se convierte en umich.edu y caret.cam.ac.uk se convierte en cam.ac.uk. Las direcciones de correo electrónico también están obligadas a usar minúsculas, y algunas de las direcciones de @ gmane.org son las siguientes:

    arwhyte-63aXycvo3TyHXe+LvDLADg@public.gmane.org
    
se convierten a la dirección real siempre que haya una dirección de correo electrónico real coincidente en otra parte del corpus de mensajes.

En la base de datos `mapping.sqlite` hay dos tablas que le permiten asignar nombres de dominio y direcciones de correo electrónico individuales que cambian durante la vida útil de la lista de correo electrónico. Por ejemplo, Steve Githens utilizó las siguientes direcciones de correo electrónico mientras cambiaba de trabajo durante la vida de la lista de desarrolladores de Sakai:

    s-githens@northwestern.edu
    sgithens@cam.ac.uk
    swgithen@mtu.edu

Podemos añadir dos entradas a la tabla de asignación de `mapping.sqlite` por lo `gmodel.py` asignará los tres a una dirección:

    s-githens@northwestern.edu ->  swgithen@mtu.edu
    sgithens@cam.ac.uk -> swgithen@mtu.edu

También puede realizar entradas similares en la tabla DNSMapping si hay varios nombres DNS que desea asignar a un solo DNS. La siguiente asignación se agregó a los datos de Sakai:

    iupui.edu -> indiana.edu

Así que todas las cuentas de los distintos campus de la Universidad de Indiana se rastrean juntas.

Puede volver a ejecutar `gmodel.py` una y otra vez a medida que mira los datos, y agregar asignaciones para hacer que los datos sean más y más limpios. Cuando haya terminado, tendrá una versión bien indexada del correo electrónico en `index.sqlite`. Este es el archivo a utilizar para hacer análisis de datos. Con este archivo, el análisis de datos será realmente rápido.

El primer análisis de datos más simple es determinar "¿quién envió más correo?" y "¿qué organización envió más correo"? Esto se hace usando `gbasic.py`:

    How many to dump? 5
    Loaded messages= 51330 subjects= 25033 senders= 1584

    Top 5 Email list participants
    steve.swinsburg@gmail.com 2657
    azeckoski@unicon.net 1742
    ieb@tfd.co.uk 1591
    csev@umich.edu 1304
    david.horwitz@uct.ac.za 1184

    Top 5 Email list organizations
    gmail.com 7339
    umich.edu 6243
    uct.ac.za 2451
    indiana.edu 2258
    unicon.net 2055

Tenga en cuenta cuánto más rápido se ejecuta `gbasic.py` en comparación con `gmane.py` o incluso `gmodel.py`. Todos están trabajando en los mismos datos, pero `gbasic.py` está utilizando los datos comprimidos y normalizados en `index.sqlite`. Si tiene muchos datos para administrar, un proceso de varios pasos como el de esta aplicación puede demorar un poco más en desarrollarse, pero le ahorrará mucho tiempo cuando realmente comience a explorar y visualizar sus datos.

Puede producir una visualización simple de la frecuencia de palabras en las líneas de asunto en el archivo `gword.py`:

    Range of counts: 33229 129
    Output written to gword.js

Esto produce el archivo `gword.js` que puede visualizar usando `gword.htm` para producir una nube de palabras similar a la del comienzo de esta sección.

Una segunda visualización es producida por `gline.py`. Calcula la participación del correo electrónico de las organizaciones a lo largo del tiempo.

    Loaded messages= 51330 subjects= 25033 senders= 1584
    Top 10 Oranizations
    ['gmail.com', 'umich.edu', 'uct.ac.za', 'indiana.edu',
    'unicon.net', 'tfd.co.uk', 'berkeley.edu', 'longsight.com',
    'stanford.edu', 'ox.ac.uk']
    Output written to gline.js

Su salida se escribe en `gline.js` que se visualiza usando `gline.htm`.

![Figura 16.4](./figures/mailorg.png)
<cennter><i>Figura 16.4: Actividad del correo Sakai por organización</i></center>

Esta es una aplicación relativamente compleja y sofisticada y tiene características para realizar una recuperación, limpieza y visualización de datos reales.

<!--NAVIGATION-->
| [Indice](indice.ipynb) | 

< [Capítulo 15 - Uso de bases de datos y SQL](cap15.ipynb) |